# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql


Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

# import data set from Ironhack's database

##CONNECTION
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'orders'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
engine = create_engine(connection_string)



#######################  WHICH TABLES WE WANT TO CHOOSE    #######################


query = 'SELECT * from orders'
orders = pd.read_sql(query,engine)
orders.head()

,index,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

**Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?**


In [9]:
customers = orders.groupby(orders.CustomerID).sum()
customers.head()

,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
12346,37126,541431,2011,1,2,10,74215,1.04,77183.60
12347,30294108,101296926,365971,1383,441,2219,2458,481.21,4310.00
12348,1900064,16869685,62324,257,111,472,2341,178.71,1797.24
12349,26148016,42165457,146803,803,73,657,631,605.10,1757.55
12350,838678,9231629,34187,34,51,272,197,65.30,334.40


**Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?**

In [31]:
per_VIP = customers.amount_spent.quantile(0.95)
print(per_VIP)

per_preferred= customers.amount_spent.quantile(0.75)
print(per_preferred)

5840.181999999983
1661.6400000000003


**Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?**

*VIP CUSTOMERS*

In [37]:
VIP = customers.loc[customers.amount_spent > per_VIP].sort_values('amount_spent', ascending = False)
print(VIP.shape)##number of VIP customer
VIP.head()


(217, 9)


,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
14646,386260126,1163267611,4182810,14191,6552,24488,197491,5176.09,280206.02
18102,98902435,243297801,866723,3746,1261,5587,64124,1940.92,259657.30
17450,66203333,188845149,677704,2292,842,4140,69993,1143.32,194550.79
16446,664956,1688629,6033,22,11,27,80997,4.98,168472.50
14911,1245078456,3196374868,11416155,46220,18930,68148,80515,26185.72,143825.06


*PREFERRED CUSTOMERS*

In [38]:
preferred = customers.loc[(customers.amount_spent > per_preferred) & (customers.amount_spent < per_VIP)].sort_values('amount_spent', ascending = False)
print(preferred.shape) ##number of preferred customer
preferred.head()


(868, 9)


,index,InvoiceNo,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,,,,,,,,,
13050,62222774,223872574,810364,2992,1278,4940,3748,1204.52,5836.86
12720,64829513,197785584,711849,2760,1106,4153,4672,956.36,5781.73
15218,28977867,92641005,333826,1009,302,1823,3329,513.44,5756.89
17686,51204459,159766169,575146,1824,905,3433,2478,1103.64,5739.46
13178,41653470,147365548,532883,1858,872,3321,3570,542.34,5725.47


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

Merge two tables with the information requested.

In [107]:
count_VIP = VIP[['amount_spent']]  #modify the VIP customers table to show just the amount_spent  (customerID is the index)

count_COUNTRY = orders[['Country','CustomerID']]      #modify the main orders table to show just the country and customerID .





,CustomerID,amount_spent
Country,,
United Kingdom,1284275187,2.197546e+09
EIRE,104468147,9.808263e+08
France,41604853,3.787897e+07
Germany,39252765,3.335796e+07
Netherlands,30463680,5.828285e+08


1. make ther merge


2. group by the Country


3. sort by #CustomerID

In [108]:
country_VIP = count_COUNTRY.merge(count_VIP,how = 'inner', on = 'CustomerID')
country_VIP = country_VIP.groupby(country_VIP.Country).sum()

country_VIP = country_VIP.sort_values('CustomerID',ascending =  False)

country_VIP.head()

,CustomerID,amount_spent
Country,,
United Kingdom,1284275187,2.197546e+09
EIRE,104468147,9.808263e+08
France,41604853,3.787897e+07
Germany,39252765,3.335796e+07
Netherlands,30463680,5.828285e+08


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

Merge the three tables with the information requested:

    1.VIP table
    
    2.Preferred table
    
    3.Orders with customersID and Country

In [120]:
count_VIP = VIP[['amount_spent']]  #modify the VIP customers table to show just the amount_spent  (customerID is the index)

count_COUNTRY = orders[['Country','CustomerID']]      #modify the main orders table to show just the country and customerID .

count_preferred = preferred[['amount_spent']]     #modify the preferred customers table to show just the amount_spent(customerID is the index)


count_preferred.head()

,amount_spent
CustomerID,
13050,5836.86
12720,5781.73
15218,5756.89
17686,5739.46
13178,5725.47


1. make ther merge Country_VIP (from the Q2) with the preferred one.



2. sort by #CustomerID

In [152]:
count_preferred = count_preferred.reset_index()

In [154]:
country_VIP = count_COUNTRY.merge(count_VIP,how = 'inner', on = 'CustomerID')

country_VIP_preferred = country_VIP.join(count_preferred ,on='CustomerID')

country_VIP_preferred




ValueError: columns overlap but no suffix specified: Index(['CustomerID', 'amount_spent'], dtype='object')

In [155]:
country_VIP

,Country,CustomerID,amount_spent
0,France,12583,7281.38
1,France,12583,7281.38
2,France,12583,7281.38
3,France,12583,7281.38
4,France,12583,7281.38
5,France,12583,7281.38
6,France,12583,7281.38
7,France,12583,7281.38
8,France,12583,7281.38
9,France,12583,7281.38
